In [18]:
import os 
os.chdir('/Users/younghun/Desktop/gitrepo/data')

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import folium

In [20]:
# 용도가 뭔지 알고 있는 라이브러리들
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings(action='ignore')
# 용도가 뭔지 몰랐던 라이브러리들
# feature 중요도를 출력하도록 지원하는 함수
import eli5


sns.set(style='whitegrid')
pd.options.display.max_columns=36
full_data = pd.read_csv('hotel_bookings.csv', encoding='utf-8')
full_data.shape

(119390, 32)

In [21]:
full_data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [22]:
# replace missing values
# agency의 결측치는 예약이 agency없이 행해진 것이라고 본다.
# company도 마찬가지
nan_replacemnts = {'country':'Unknown', 'agent':0,
                  'children':0.0, 'company':0}
full_data = full_data.fillna(nan_replacemnts)

In [23]:
# meal변수의 Undefined값은 SC값과 동일하다.
full_data['meal'] = full_data['meal'].replace('Undefined','SC')

**몰랐던 방법 1**

- index를 활용해서 연산량을 줄이자!

In [24]:
# adults, children, babies가 모두 0인 행들을 삭제하자.
# 그러므로 위의 3가지 value들의 합이 0인 데이터들의 index를 활용해 삭제! -good!
zero_idx = full_data.loc[full_data['adults']+full_data['children']
             +full_data['babies'] == 0].index
full_data_cln = full_data.drop(zero_idx, axis=0)
print("제거 전 shape:", full_data.shape)
print("제거 후 shape:", full_data_cln.shape)
print("adults, children, babies가 모든 0인 데이터 개수: ",
     full_data.shape[0] - full_data_cln.shape[0])

제거 전 shape: (119390, 32)
제거 후 shape: (119210, 32)
adults, children, babies가 모든 0인 데이터 개수:  180


In [25]:
# 이제 리조트, 시티호텔 구분하기
# 호텔 종류별 방문객 수 살펴보기 위해서 취소되지 않은 예약 데이터들만 남기기
rh = full_data_cln.loc[(full_data_cln['hotel']=='Resort')
                      &(full_data_cln['is_canceled']==0)]
ch = full_data_cln.loc[(full_data_cln['hotel']=='City')
                      &(full_data_cln['is_canceled']==0)]

- 특정 조건을 만족하는 데이터들 중 특정 변수들 값을 count하기!
- df.loc[df['column']==조건]['column2'].value_counts()

In [41]:
# 나라별 호텔 손님 수 데이터프레임 만들기
country_count = pd.DataFrame(full_data_cln.loc[full_data_cln['is_canceled']==0]['country'].value_counts())
country_count.rename(columns={'country':'number of guests'},
                    inplace=True)
# 모든 나라의 총 손님수를 각 나라의 손님수로 나누어서 비율로 나타내기
total_guests = country_count['number of guests'].sum()
country_count['Ratio of guests'] = round(country_count['number of guests'] / total_guests, 4)
country_count = country_count.reset_index()
country_count.rename(columns={'index':'country'},
                    inplace=True)
country_count.head()

,country,number of guests,Ratio of guests
0,PRT,20977,0.2797
1,GBR,9668,0.1289
2,FRA,8468,0.1129
3,ESP,6383,0.0851
4,DEU,6067,0.0809


- plotly.express 표현하기
- 추가 레이아웃 꾸미는 건 update_traces로!

In [45]:
# pie graph 로 나타내보기
# plotly로 나타내기
fig = px.pie(country_count, values='number of guests',
      names='country', title='Home country of guests',
      template='seaborn')
# 추가적인 레이아웃 꾸미기는 update_traces로!
fig.update_traces(textposition='inside',
                 textinfo='value+percent+label')
fig.show()

In [48]:
# plotly.express는 choropleth를 사용할 수 있음!
# 손님 비율을 color로 설정
# hover_name은 folium의 tooltip같은 역할
guest_map = px.choropleth(country_count,
             locations=country_count['country'],
             color=country_count['Ratio of guests'],
             hover_name=country_count['country'],
             color_continuous_scale=px.colors.sequential.Plasma,
             title='Home country of guests')
guest_map.show()

In [49]:
# 하루 밤 묵는데 얼마나 돈을 쓰는지에 대한 지표는 adr 변수이다..!
# adr 변수 정규화시켜주기
full_data_cln['adr_pp'] = full_data_cln['adr'] / (full_data_cln['adults']
                                                 + full_data_cln['children'])
full_data_guests = full_data_cln.loc[full_data_cln['is_canceled']==0]


0           0.00
1           0.00
2          75.00
3          75.00
4          98.00
           ...  
119385     96.14
119386    225.43
119387    157.71
119388    104.40
119389    151.20
Name: adr, Length: 119210, dtype: float64